In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score

import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

In [0]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from keras.preprocessing.image import random_rotation, random_shift, random_zoom
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.models import Model, Sequential, model_from_json
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils

In [0]:
pwd

'/content'

In [0]:
os.chdir('/content')

In [0]:
# !mkdir .kaggle
# !ls -a
# import json

# token = {"username":"ryohanoi","key":"21565eed4a7fe5fa312665088bdf2d20"}
# with open('/content/.kaggle/kaggle.json', 'w') as file:
#     json.dump(token, file)

# !!chmod 600 /content/.kaggle/kaggle.json
# !ls -a .kaggle
# !mkdir　/root/.kaggle/
# !cp -R /content/.kaggle/kaggle.json /root/.kaggle/
# !ls -a /root/.kaggle/
# !kaggle competitions list

In [0]:
# !mkdir tgs-salt-identification-challenge

In [0]:
# os.chdir('/content/drive/My Drive/Colab Notebooks')
# !kaggle competitions download -c tgs-salt-identification-challenge

In [0]:
# os.chdir('/content/drive/My Drive/Colab Notebooks/Salt')
# !unzip competition_data.zip -d competition_data

In [0]:
# os.chdir('/content/drive/My Drive/Colab Notebooks')
# !git clone https://github.com/zhixuhao/unet.git

# セグメンテーション

KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行います。


[TGS Salt Identification Challenge | Kaggle](https://www.kaggle.com/c/tgs-salt-identification-challenge)


セグメンテーション手法としてU-Net[1]を使います。


[1]O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015


https://arxiv.org/pdf/1505.04597.pdf




## 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。


  [zhixuhao/unet: unet for image segmentation](https://github.com/zhixuhao/unet)


**《GPU環境での学習》**


大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。


**《新たなデータセットの適用》**


公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。



In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks/unet')
!python main.py

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/drive/My Drive/Colab Notebooks/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


2020-03-23 07:17:51.765268: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-23 07:17:51.765641: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x28d4bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-23 07:17:51.765678: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-23 

## 【問題2】コードリーディング
論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

* Unetは画像の座標情報を保持するために全結合NN層を用いないFCNの考えを基にしている。
* FCNと比較した際にUnetは元の画像の情報を反映させるため下記のような構造になっている。

model.py

```
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
```